<a href="https://colab.research.google.com/github/arnaudstdr/films_reco/blob/main/Films_recommandations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Système de recommandations de Films
Développement d'un système de recommandation de films basé sur l'analyse du contenu utilisant le Natural Language Processing (NLP) et des algorithmes de similarité vectorielle.

## Sommaire

- <a href="#importation">1. Importation des outils</a>
- <a href="#chargement-donnees">2. Chargement des données</a>
- <a href="#preparation-donnees">3. Préparation des données</a>
- <a href="#representation-vecto">4. Représentation vectiorielle</a>
- <a href="#similarité">5. Calcul de la similarité</a>
- <a href="#fonction-reco">6. Fonctions de recommandation</a>
- <a href="#visualisation">7. Visualisation</a>
- <a href="#evaluation">8. Évaluation du système</a>


## 1. <a id="importation">Importation des outils</a>



In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns

## 2. <a id="chargement-donnees">Chargement des données</a>

Télécharger les données depuis https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROJETS_ML/Films_recommandations/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROJETS_ML/Films_recommandations/tmdb_5000_credits.csv')

credits.head()

In [ ]:
movies.head()

## 3. <a id="preparation-donnees">Préparation des données</a>

In [ ]:
# Modification de la colonne 'movie_id' de credits en 'id' pour faciliter la fusion
credits.rename(columns={'movie_id': 'id'}, inplace=True)

credits.head()

#### Fusion des deux DataFrames
La fusion des deux dataframes facilite l'analyse et la création du modèle :
- Toutes les informations nécessaire sont disponible dans un seul DataFrame.
- Permet de manipuler les données plus efficacement et d'extraire les caractéristiques pertinentes.

In [ ]:
# Fusion des DataFrames
movies_data = movies.merge(credits, on='id')

movies_data.head()

- Les deux DataFrames contiennent une colones `title`, alors lors de la fusion Pandas ajoute des suffixes `_x` et `_y` pour éviter les conflits.
- Comme les colonnes `title_x` et `title_y` sont identiques, nous allons n'en garder qu'un seul.  

In [ ]:
movies_data = movies_data.drop(columns=['title_y'])
movies_data.rename(columns={'title_x': 'title'}, inplace=True)

movies_data.head()

In [ ]:
# Sélection des caractéristiques pertinentes
features = ['title', 'overview', 'genres', 'keywords', 'cast', 'crew']
movies_features = movies_data[features]

**Le nettoyage des valeurs NaN (valeurs manquantes)** est une étape essentielle pour éviter les erreurs et garantir. Pourquoi ?
1. Éviter les erreurs lors des calculs
  - `TfidfVectorizer` ou `cosine_similarity` exigent des entrées complètes.
2. Améliorer la qualité des recommandations
  - Une ligne contenant des `Nan` peut fausser les prédictions.
3. Préparer les données pour l'entraînement du modèle

In [ ]:
# Nettoyage des données NaN
movies_features = movies_features.fillna('')

#### Conversion des colonnes JSON en chaînes de caractères
- Facilite l'analyse et la manipulation
- Rend compatible avec les algorithmes de NLP
- Permet un affichage plus simple et lisible

In [ ]:
# Conversion des colonnes JSON en chaînes de caractères
import ast

def convert_json(obj):
  try:
    result = ast.literal_eval(obj)
    names = [item['name'] for item in result]
    return ' '.join(names)
  except:
    return ''

movies_features['genres'] = movies_features['genres'].apply(convert_json)
movies_features['keywords'] = movies_features['keywords'].apply(convert_json)

In [ ]:
# Pour les acteurs, on ne garde que les 3 premiers
def get_top_actors(cast_json):
  try:
    cast = ast.literal_eval(cast_json)
    top_cast = [actor['name'] for actor in cast[:3]]
    return ' '.join(top_cast)
  except:
    return ''

movies_features['cast'] = movies_features['cast'].apply(get_top_actors)

In [ ]:
# Pour l'équipe, on ne garde que le réalisateur
def get_director(crew_json):
  try:
    crew = ast.literal_eval(crew_json)
    directors = [member['name'] for member in crew if member['job'] == 'Director']
    return ' '.join(directors)
  except:
    return ''

## 4. <a id="representation-vecto">Création de la représentation vectorielle des films</a>
Il est nécessaire de créer une représentation vectorielle pour :
- Transformer du texte en nombres pour que les algorithmes puissent l'utiliser
- Comparer les films entre eux grâce à la similarité cosinus
- Créer un moteur de recommandations

In [ ]:
# Concaténation de toutes les caractéristiques
movies_features['combined_features'] = (
    movies_features['overview'] + ' ' +
    movies_features['genres'] + ' ' +
    movies_features['keywords'] + ' ' +
    movies_features['cast'] + ' ' +
    movies_features['crew']
)

In [ ]:
# Vectorisation des caractéristiques avec TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_features['combined_features'])

## 5. <a id="similarité">Calcul de la similarité</a>

In [ ]:
# Utilisation de la similarité cosinus
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

## 6. <a id="fonction-reco">Fonction de recommandation</a>

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim, movies_df=movies_features):
  # Vérifie si le film existe dans le base de données
  if title not in movies_df['title'].values:
    close_matches = movies_df[movies_df['title'].str.contains(title, case=False)]
    if not close_matches.empty:
      title = close_matches.iloc[0]['titel']
      print(f"Film introuvable. Utilisation de '{title}' à la place.")
    else:
      return "Film introuvable. Veuillez essayer un autre titre."

  # Récupération de l'index du film
  idx = movies_df[movies_df['title'] == title].index[0]

  # Création d'une liste de tuples (positions, score de similarité)
  sim_scores = list(enumerate(cosine_sim[idx]))

  # Tri des films en fonction du score de similarité
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # Top 10 des films similaires (en excluant le film lui-même)
  sim_scores = sim_scores[1:11]

  # Récupération des indices des films
  movie_indices = [i[0] for i in sim_scores]

  # Retourner les 10 films les plus similaires
  recommanded_movies = movies_df['title'].iloc[movie_indices]

  # Récupérer les scores pours visualisation
  similarity_scores = [i[1] for i in sim_scores]

  # Créer un DataFrame avec les recommandations et leurs scores
  recommandations = pd.DataFrame({
      'Film': recommanded_movies.values,
      'Score de similarité': similarity_scores
  })

  return recommandations

## 7. <a id="visualisation">Visualisation</a>

In [ ]:
def plot_recommandations(recommandations):
  plt.figure(figsize=(10, 6))

  # Créer un graphique à barres horizontales
  sns.barplot(x='Score de similarité', y='Film', data=recommandations)

  plt.title('Films recommandés et leurs scores de similarité')
  plt.xlabel('Score de similarité')
  plt.ylabel('Films')
  plt.tight_layout()

  plt.savefig('recommandations_plot.png')
  plt.show()

## 8. <a id="evaluation">Évaluation du sytème</a>

In [ ]:
def evaluate_system():
  # Sélectionner des films populaires l'évaluation
  popular_movies = movies_data.sort_values('popularity', ascending=False).head(10)['title'].values

  results = {}
  for movie in popular_movies:
    recommandations = get_recommendations(movie)
    if isinstance(recommandations, pd.DataFrame):
      # Calculer le score moyen de similarité
      avg_score = recommandations['Score de similarité'].mean()
      results[movie] = avg_score

  # Visualiser les résultats
  plt.figure(figsize=(12, 6))
  plt.bar(results.keys(), results.values())
  plt.xticks(rotation=45, ha='right')
  plt.title('Score moyen de similarité pour différents films populaires')
  plt.ylabel('Score moyen')
  plt.tight_layout()

  plt.savefig('evaluation_plot.png')
  plt.show()

  return results

In [ ]:
# Exemple d'utilisation
if __name__ == "__main__":
  # Exemple : recommandations pour "The Dark Knight"
  recommandations = get_recommendations("The Dark Knight")
  print(recommandations)

  # Visualisation des recommandations
  plot_recommandations(recommandations)

  # Évaluation du sytème
  evaluation_results = evaluate_system()
  print("\nRésultats de l'évaluation :")
  for movie, score in evaluation_results.items():
    print(f"{movie}: {score:.4f}")